In [6]:
# ! langchain_community


In [6]:
from langchain.document_loaders import TextLoader

In [7]:
loader = TextLoader("file.txt")
data = loader.load()

In [11]:
data[0].metadata

{'source': 'file.txt'}

In [14]:
# !pip3 install unstructured libmagic python-magic 
# python-magic-bin

In [15]:
from langchain.document_loaders import UnstructuredURLLoader

In [17]:
loader = UnstructuredURLLoader(urls=[
    "https://www.cricbuzz.com/profiles/1413/virat-kohli",
    "https://www.cricbuzz.com/profiles/25/sachin-tendulkar"
])

In [18]:
data = loader.load()
len(data)

2

In [21]:
data[1].metadata

{'source': 'https://www.cricbuzz.com/profiles/25/sachin-tendulkar'}

In [23]:
text = """
Rashmika Mandanna (/rəʃmɪkɑː mənðənɑː/; born 5 April 1996) is an Indian actress who predominantly works in Telugu, Kannada and Hindi films. She has received four SIIMA Awards and a Filmfare Award. One of South India's highest-paid actresses, she was featured in Forbes India's 2024 list of "30 Under 30".

Rashmika made her acting debut with the Kannada romantic comedy Kirik Party (2016), and found further success in the action picture Anjani Putra and romance Chamak (both 2017). Her first Telugu film release came in the romantic comedy Chalo but the romance Geetha Govindam (both 2018) proved to be a turning point in her career and earned her the Filmfare Critics Award for Best Actress – Telugu. She later played the leading lady in the Telugu action comedies Devadas (2018) and Sarileru Neekevvaru (2020), and the romance Bheeshma (2020). She had her first Tamil release in the action drama Sulthan (2021), and later achieved her top-grosser in the language with the action film Varisu (2023), though her inclination towards androcentric films drew criticism for providing only limited scope to showcase her acting abilities.

Rashmika earned wider recognition as a head-strong girl in the action film Pushpa: The Rise (2021). After playing a supporting role in the Telugu period drama Sita Ramam (2022), she expanded to Hindi cinema and had her highest-grossing release with the action drama Animal (2023).

In addition to acting, Rashmika serves as the ambassador of several brands and products.

Early and personal life
Rashmika Mandanna was born on 5 April 1996 to Suman and Madan Mandanna into a Kodava Hindu family in Virajpet, a town in Kodagu district, Karnataka.[1][2] Her father owns a coffee estate and a function hall in her hometown, and her mother is a homemaker.[3] She has a younger sister, Shiman.[4] As a child, Rashmika's family struggled financially, and had difficulties finding a home and paying rent; according to Rashmika, her parents could not afford to buy her toys and the experience has continued to influence her relationship with money.[5]

Rashmika attended Coorg Public School, a boarding school in Gonikoppal.[6] During this phase of her life, Rashmika was often misunderstood due to her struggles with communication, making connecting with her peers difficult.[7] Rashmika credits her mother for being stabilising influence in her life and labelled her "my biggest strength".[7]

Rashmika studied for a bachelor's degree in Psychology, Journalism and English Literature at M. S. Ramaiah College of Arts, Science and Commerce in Bangalore.[8] Despite her parents' initial hesitation towards Rashmika's pursuit of an acting career due to her unassertive personality, her parents granted her permission to go ahead with her plans.[9] In 2014, Rashmika was awarded The Times of India's Clean & Clear Fresh Face that recognizes individuals with the greatest potential for future success;[10] the award was presented to her by Akshay Kumar and she was subsequently made a brand ambassador for Clean & Clear as part of the winning reward.[11][12] This was followed by a brief career in modelling, which Rashmika believed would ease her transition into acting.[12]

Rashmika began dating Rakshit Shetty, her co-star in Kirik Party, during the making of the film, and they announced their engagement on 3 July 2017 at a private party in her hometown Virajpet.[13] They ended their engagement in September 2018, citing compatibility issues.[14]
"""

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n",".", " "],
    chunk_size=200,
    chunk_overlap=0
)

chunks = r_splitter.split_text(text)

In [31]:
## Our Vector Database

In [30]:
# for chunk in chunks:
#     print(len(chunk))



In [59]:
# ! pip install faiss-cpu
# ! pip install sentence-transformers
# !pip install openai
# !pip install -U langchain-cohere
import getpass
import os
os.environ["COHERE_API_KEY"] = getpass.getpass()



 ········


In [62]:
import os
# import streamlit as st
import pickle
import time
import langchain
# from langchain import OpenAI
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQAWithSourcesChain

from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
# from langchain.embeddings import OpenAIEmbeddings
from langchain_cohere import CohereEmbeddings
from langchain.vectorstores import FAISS

In [64]:
# llm = OpenAI(temprature = 0.9,max_tokens = 500)
llm = Cohere(max_tokens=256, temperature=0.75)
loader = UnstructuredURLLoader(urls=[
    "https://www.cricbuzz.com/profiles/1413/virat-kohli",
    "https://www.cricbuzz.com/profiles/25/sachin-tendulkar"
])
data = loader.load()

len(data)

NameError: name 'Cohere' is not defined

In [48]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0
)

docs=r_splitter.split_documents(data)
len(docs)

478

In [50]:
docs[9]

Document(metadata={'source': 'https://www.cricbuzz.com/profiles/1413/virat-kohli'}, page_content='15th Match, Group B\n\nICC Mens East Asia-Pacific T20WCQA\n\nVanuatu vs Fiji\n\n9th Match')

In [53]:
# !pip install tiktoken

In [63]:
# embeddings = OpenAIEmbeddings()
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)
vectorindex_openai = FAISS.from_documents(docs,embeddings)

In [71]:
vectorindex_openai

In [74]:
# import pickle
# file_path = "vector_index.pkl"
# with open(file_path,"wb") as f:
#     pickle.dump(vectorindex_openai,f)

In [72]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm,retriever=vectorindex_openai.as_retriever())

In [73]:
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatCohere(client=<cohere.client.Client object at 0x7f7fbffff010>, async_client=<cohere.client.AsyncClient object at 0x7f7fbffff6d0>, cohere_api_key=SecretStr('**********'))), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.

In [78]:
query = "what is special about Virat Kohli?"
langchain.debug = True

In [80]:
chain({"question":query},return_only_outputs=True)